In [2]:
import pandas as pd

In [3]:
input_file = "PT OM change scores - anonymous.xlsx"
df = pd.read_excel(input_file)

In [4]:
print("Columns in dataset:", df.columns)

Columns in dataset: Index(['#', 'BBS ADM', '10MWT SS ADM', '10MWT FS ADM', '6MWT ADM',
       'MiniBESTest ADM', 'SPPB ADM', 'BBS W1', 'BBS CHANGE ADM-W1',
       '10MWT SS W1',
       ...
       '10MWT SS DC', '10MWT SS CHANGE ADM-DC', '10MWT FS DC',
       '10MWT FS CHANGE ADM-DC', '6MWT DC', '6MWT CHANGE ADM-DC',
       'MiniBESTest DC', 'MiniBESTest CHANGE ADM-DC', 'SPPB DC',
       'SPPB CHANGE ADM-DC'],
      dtype='object', length=175)


In [6]:
print("Number of rows: %d, and number of columns: %d" % df.shape)

Number of rows: 201, and number of columns: 175


In [7]:
df.head()

,#,BBS ADM,10MWT SS ADM,10MWT FS ADM,6MWT ADM,MiniBESTest ADM,SPPB ADM,BBS W1,BBS CHANGE ADM-W1,10MWT SS W1,...,10MWT SS DC,10MWT SS CHANGE ADM-DC,10MWT FS DC,10MWT FS CHANGE ADM-DC,6MWT DC,6MWT CHANGE ADM-DC,MiniBESTest DC,MiniBESTest CHANGE ADM-DC,SPPB DC,SPPB CHANGE ADM-DC
0,1.0,46,0.88,1.14,324,17,NaN,49.0,3,1,...,1.123596,0.243596,1.481481,0.341481,460,136,23,6,-,-
1,2.0,48,0.86,1.56,468,11,NaN,50.0,2,1.34,...,1.2,0.34,1.8,0.24,506,38,16,5,-,-
2,3.0,30,0.22,0.36,113,4,NaN,NaN,NaN,NaN,...,0.34965,0.12965,0.420168,0.060168,125,12,9,5,-,-
3,4.0,1,0,0,0,0,NaN,1.0,0,0,...,0.37237,0.37237,-,-,66,66,0,0,-,-
4,5.0,42,0.73,0.86,210,10,NaN,44.0,2,0.5,...,0.632911,-0.097089,0.766284,-0.093716,212,2,14,4,-,-


In [8]:
df.describe()

,#,BBS W1,SPPB CHANGE ADM-W1,BBS CHANGE W1-W2,10MWT SS CHANGE W1-W2,10MWT FS CHANGE W1-W2,6MWT CHANGE W1-W2,MiniBESTest CHANGE W1-W2,SPPB CHANGE W1-W2,BBS W3,...,10MWT SS W13,10MWT SS CHANGE W12-W13,10MWT FS W13,10MWT FS CHANGE W12-W13,6MWT W13,6MWT CHANGE W12-W13,MiniBESTest W13,MiniBESTest CHANGE W12-W13,SPPB W13,SPPB CHANGE W12-W13
count,200.000000,156.000000,117.000000,107.000000,107.000000,100.000000,101.000000,106.000000,81.000000,83.000000,...,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
mean,100.500000,32.942308,0.829060,3.121495,0.069158,0.092471,22.633663,1.264151,0.419753,29.337349,...,0.539665,0.079366,0.524109,-0.085833,173.0,-6.0,6.0,5.0,5.0,4.0
std,57.879185,16.288623,1.428108,3.939903,0.109496,0.146151,35.359503,2.015662,1.404139,15.638662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,0.000000,-3.000000,-7.000000,-0.205857,-0.370143,-89.000000,-3.000000,-6.000000,1.000000,...,0.539665,0.079366,0.524109,-0.085833,173.0,-6.0,6.0,5.0,5.0,4.0
25%,50.750000,21.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,...,0.539665,0.079366,0.524109,-0.085833,173.0,-6.0,6.0,5.0,5.0,4.0
50%,100.500000,37.000000,1.000000,2.000000,0.047407,0.079141,20.000000,1.000000,0.000000,34.000000,...,0.539665,0.079366,0.524109,-0.085833,173.0,-6.0,6.0,5.0,5.0,4.0
75%,150.250000,46.000000,2.000000,5.000000,0.142658,0.164197,47.000000,2.000000,1.000000,43.000000,...,0.539665,0.079366,0.524109,-0.085833,173.0,-6.0,6.0,5.0,5.0,4.0
max,200.000000,56.000000,7.000000,17.000000,0.317045,0.478240,115.000000,7.000000,4.000000,52.000000,...,0.539665,0.079366,0.524109,-0.085833,173.0,-6.0,6.0,5.0,5.0,4.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Columns: 175 entries, # to SPPB CHANGE ADM-DC
dtypes: float64(139), object(36)
memory usage: 274.9+ KB


In [10]:
df.isnull().sum()

#                            1
BBS ADM                      2
10MWT SS ADM                 2
10MWT FS ADM                 2
6MWT ADM                     2
                            ..
6MWT CHANGE ADM-DC           2
MiniBESTest DC               2
MiniBESTest CHANGE ADM-DC    2
SPPB DC                      2
SPPB CHANGE ADM-DC           2
Length: 175, dtype: int64